[![Colab Badge](https://img.shields.io/badge/Open_in_Colab-blue?style=for-the-badge)][colab-link]
<a href="javascript:void(0);" onclick="openJupyterWidget('https://github.com/nmfs-opensci/nmfshackdays-2025/blob/main/topics-2025/2025-opendap/3-earthdata-auth.ipynb');">
    <img src="https://img.shields.io/badge/Open_in_JupyterHub-orange?style=for-the-badge" alt="JupyterHub Badge">
</a> [![Download Badge](https://img.shields.io/badge/Download-grey?style=for-the-badge)][download-link]

[download-link]: https://nmfs-opensci.github.io/NMFSHackDays-2025/topics-2025/2025-opendap/3-earthdata-auth.ipynb
[colab-link]: https://colab.research.google.com/github/nmfs-opensci/nmfshackdays-2025/blob/main/topics-2025/2025-opendap/3-earthdata-auth.ipynb
[jupyter-link]: https://nmfs-openscapes.2i2c.cloud/hub/user-redirect/lab?fromURL=https://raw.githubusercontent.com/nmfs-opensci/nmfshackdays-2025/main/topics-2025/2025-opendap/3-earthdata-auth.ipynb

## Overview

The National Snow and Ice Data Center servers require NASA Earthdata login authentication, but don't require a EULA (as far as I know). However they have similar redirect issues as data that does require a EULA. The solution used for datasets that require a EULA seems to work for access.

I suggest going throuhg 3-nasa.ipynb before this tutorial.

### Prerequisites

I assume you have a `.netrc` file at `~` (home). `~/.netrc` should look just like this with your username and password.

```
machine urs.earthdata.nasa.gov
        login yourusername
        password yourpassword
```

### Packages and setup


In [1]:
import xarray as xr
import pydap.client

In [2]:
# create an authenticated session
import earthaccess
earthaccess.login()
edl_session = earthaccess.get_requests_https_session()

## National Snow and Ice Data Center

Their OPeNDAP server also uses NASA Earthdata login authentication. I struggled to find the right format for url that would open.

In [3]:
#url = "https://n5eil02u.ecs.nsidc.org/opendap/OTHR/NISE.004/2012.10.02/NISE_SSMISF17_20121002.HDFEOS"
url = "https://n5eil02u.ecs.nsidc.org/opendap/MOST/MOD10A1.006/2000.03.22/MOD10A1.A2000082.h00v08.006.2016061212345.hdf"

In [ ]:
dap4_url = url.replace("https", "dap4", 1)
ds = xr.open_dataset(dap4_url, engine="pydap", session=edl_session)
ds

In [5]:
import pydap
from pydap.client import open_url
from pydap.cas.urs import setup_session
session = setup_session(username, password, check_url=url)

In [6]:
pydap_ds = open_url(url, session=session, protocol="dap2")

In [5]:
ds = xr.open_dataset(url, session=edl_session, engine="pydap")

/srv/conda/envs/notebook/lib/python3.12/site-packages/pydap/handlers/dap.py:123: UserWarning: PyDAP was unable to determine the DAP protocol defaulting to DAP2 which is consider legacy and may result in slower responses. For more, see go to https://www.opendap.org/faq-page.
  _warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:200: SerializationWarning: variable 'NDSI_Snow_Cover' has multiple fill values {np.int64(200), np.int64(255)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:200: SerializationWarning: variable 'Snow_Albedo_Daily_Tile' has multiple fill values {np.int64(250), np.int64(255)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [46]:
pydap_ds

<DatasetType with children 'NDSI_Snow_Cover', 'NDSI_Snow_Cover_Basic_QA', 'NDSI_Snow_Cover_Algorithm_Flags_QA', 'NDSI', 'Snow_Albedo_Daily_Tile', 'orbit_pnt', 'granule_pnt', 'Latitude', 'Longitude', 'YDim', 'XDim', 'MOD_Grid_Snow_500m_eos_cf_projection'>

In [7]:
%%time
# this works but odd errors
store = xr.backends.PydapDataStore(pydap_ds)
ds = xr.open_dataset(store)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:200: SerializationWarning: variable 'NDSI_Snow_Cover' has multiple fill values {np.int64(200), np.int64(255)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:200: SerializationWarning: variable 'Snow_Albedo_Daily_Tile' has multiple fill values {np.int64(250), np.int64(255)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


CPU times: user 604 ms, sys: 44.7 ms, total: 649 ms
Wall time: 1.18 s


In [9]:
%%time
url="https://n5eil02u.ecs.nsidc.org/opendap/MOST/MOD10A1.006/2000.03.22/MOD10A1.A2000082.h00v08.006.2016061212345.hdf"
ds = xr.open_dataset(url, engine="pydap", session=session)

/srv/conda/envs/notebook/lib/python3.12/site-packages/pydap/handlers/dap.py:123: UserWarning: PyDAP was unable to determine the DAP protocol defaulting to DAP2 which is consider legacy and may result in slower responses. For more, see go to https://www.opendap.org/faq-page.
  _warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:200: SerializationWarning: variable 'NDSI_Snow_Cover' has multiple fill values {np.int64(200), np.int64(255)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/conventions.py:200: SerializationWarning: variable 'Snow_Albedo_Daily_Tile' has multiple fill values {np.int64(250), np.int64(255)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


CPU times: user 55 ms, sys: 15.3 ms, total: 70.2 ms
Wall time: 1.09 s


## Conclusion

Working with NSIDC data on its OPeNDAP server is hard. Redirect issues though I can open files with xarray. But format is odd.

## References

* <https://github.com/pydap/pydap/issues/188>
* <https://nsidc.org/data/user-resources/help-center/how-do-i-access-data-using-opendap#anchor-using-a-command-line-interface>

